# CUPED and CUPAC Tutorial

This tutorial demonstrates variance reduction techniques for A/B testing using covariate adjustment methods available in HypEx.

**CUPED** (Controlled Experiments Using Pre-Experiment Data) uses historical features to reduce variance in your target metrics through linear regression adjustment.

**CUPAC** (Covariate-Updated Pre-Analysis Correction) extends CUPED by using multiple pre-experiment covariates to predict pre-experiment target values, then subtracting these predictions from current experiment targets. This approach supports different regression models (linear, ridge, lasso, catboost) and avoids data leakage by never using experiment data to predict experiment outcomes.

Both methods help you:
- Detect smaller effects with the same sample size
- Reduce sample size needed to detect the same effect
- Increase statistical power of your experiments

## Table of Contents
<ul>
  <li><a href="#data-preparation">Data Preparation</a></li>
  <li><a href="#baseline-ab-test">Baseline AB Test</a></li>
  <li><a href="#cuped-implementation">CUPED Implementation</a></li>
  <li><a href="#cupac-implementation">CUPAC Implementation</a></li>
  <li><a href="#best-practices">Best Practices</a></li>
</ul>

## Data Preparation

For CUPAC to work correctly with the new features_mapping format, we need:
1. **Target metrics**: The metrics you want to analyze (e.g., spends, revenue)
2. **Historical target features**: Lagged versions of your targets from different time periods
3. **Pre-experiment covariates**: Features measured before the experiment that correlate with outcomes

The new CUPAC implementation supports **multilevel models** - it can automatically create models for each available time period transition. For this tutorial, we'll use **2 time periods**:
- Period 2 → Period 1: `y0_lag_2 ~ X1_lag2 + X2_lag2`  
- Period 1 → Current: `y0_lag_1 ~ X1_lag1 + X2_lag1`

Each period uses its own set of covariates, making the temporal structure clearer.

Let's generate synthetic data using the built-in DataGenerator:

In [1]:
from hypex import ABTest
from hypex.dataset import Dataset, InfoRole, TargetRole, TreatmentRole, FeatureRole
from hypex.utils.tutorial_data_creation import DataGenerator

In [2]:
# Generate synthetic data with 2 historical periods using built-in DataGenerator
gen = DataGenerator(
    n_samples=2000,
    distributions={
        "X1": {"type": "normal", "mean": 0, "std": 1},
        "X2": {"type": "bernoulli", "p": 0.5},
        "y0": {"type": "normal", "mean": 5, "std": 1},
    },
    time_correlations={"X1": 0.2, "X2": 0.1, "y0": 0.6},
    effect_size=2.0,
    seed=42
)

df = gen.generate()
# Keep only the columns we need for 2-period CUPAC
df = df.drop(columns=['y0', 'z', 'U', 'D', 'y1'])
df = df.rename(columns={'y0_lag_1': 'y_lag1', 'y0_lag_2': 'y_lag2'})

In [3]:
data = Dataset(
    roles = {
    "d": TreatmentRole(),
    "y": TargetRole(cofounders=["X1", "X2"]),

    "y_lag1": TargetRole(parent="y", lag=1),
    "X1_lag1": FeatureRole(parent="X1", lag=1),
    "X2_lag1": FeatureRole(parent="X2", lag=1),

    "y_lag2": TargetRole(parent="y", lag=2),
    "X1_lag2": FeatureRole(parent="X1", lag=2),
    "X2_lag2": FeatureRole(parent="X2", lag=2),
    },
    data=df,
    default_role=InfoRole(),
)

## Baseline AB Test

First, let's run a standard AB test without any variance reduction to establish our baseline:

In [4]:
# Standard AB test without covariate adjustment
test_baseline = ABTest()
result_baseline = test_baseline.execute(data)

result_baseline.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.737214,7.961045,3.223832,68.05333,OK,1.012032e-184


In [5]:
result_baseline.sizes

,control size,test size,control size %,test size %,group
1,1328,672,66.4,33.6,1


## CUPED Implementation

CUPED uses a single historical feature to adjust the target variable. In HypEx, specify the `cuped_features` parameter:

**Note**: For this dataset, we'll use the period 1 lagged features for CUPED since it's the closest to the current target.

In [6]:
# CUPED with single covariate (using closest lagged feature)
test_cuped = ABTest(cuped_features={'y': 'y_lag1'})
result_cuped = test_cuped.execute(data)

result_cuped.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.737214,7.961045,3.223832,68.053330,OK,1.012032e-184
1,y_cuped,1,4.851992,7.734221,2.882229,59.403002,OK,1.740353e-213


In [7]:
# Check variance reduction achieved by CUPED
result_cuped.variance_reduction_report

,Transformed Metric Name,Variance Reduction (%)
0,y_cuped,28.79786


## CUPAC Implementation

The new CUPAC implementation uses `features_mapping` format and automatically creates multilevel models. The `features_mapping` is already configured in our Dataset above.

Key advantages of the new multilevel approach:
- **Sequential modeling**: Each time period predicts the next period
- **Better temporal relationships**: Captures changing correlations over time  
- **Multiple targets**: Different targets can have different numbers of periods
- **Automatic model selection**: Chooses best performing models via cross-validation

**Example with 3 periods**: For more complex scenarios, you can use 3 or more periods:
- Period 3 → Period 2: `target_lag_3 ~ covariates_lag3`
- Period 2 → Period 1: `target_lag_2 ~ covariates_lag2`  
- Period 1 → Current: `target_lag_1 ~ covariates_lag1`

In [8]:
# Multilevel CUPAC with linear regression
test_cupac_linear = ABTest(
    enable_cupac=True,
    cupac_models='linear'
)
result_cupac_linear = test_cupac_linear.execute(data)

result_cupac_linear.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.737214,7.961045,3.223832,68.05333,OK,1.012032e-184
1,y_cupac,1,5.711284,8.457811,2.746527,48.08949,OK,7.564866e-212


In [9]:
# Multilevel CUPAC with ridge regression
test_cupac_ridge = ABTest(
    enable_cupac=True,
    cupac_models='ridge'
)
result_cupac_ridge = test_cupac_ridge.execute(data)

result_cupac_ridge.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.737214,7.961045,3.223832,68.053330,OK,1.012032e-184
1,y_cupac,1,5.711252,8.457863,2.746610,48.091209,OK,7.484948e-212


In [10]:
# Multilevel CUPAC with automatic model selection
test_cupac_auto = ABTest(
    enable_cupac=True,
    cupac_models=['linear', 'ridge', 'lasso']  # Will select best performing model for each transition
)
result_cupac_auto = test_cupac_auto.execute(data)

result_cupac_auto.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.737214,7.961045,3.223832,68.05333,OK,1.012032e-184
1,y_cupac,1,5.711284,8.457811,2.746527,48.08949,OK,7.564866e-212


In [11]:
# Check variance reduction for CUPAC methods
result_cupac_ridge.variance_reductions

,target,best_model,variance_reduction_cv,variance_reduction_real
0,y,ridge,64.045831,34.949871


## Best Practices

### New Features_Mapping Format

The new CUPAC implementation uses `features_mapping` in the Dataset instead of `cupac_features` in ABTest:

```python
# NEW FORMAT (current implementation)
data = Dataset(
    roles={"d": TreatmentRole(), "y": TargetRole()},
    data=df,
    features_mapping={
        "y": {
            ("y0_lag_1", 1): ["X1_lag1", "X2_lag1"],    # Period 1 → Current
            ("y0_lag_2", 2): ["X1_lag2", "X2_lag2"],    # Period 2 → Period 1
        }
    }
)
test = ABTest(enable_cupac=True, cupac_model='linear')
```

### Extending to More Time Periods

You can easily extend this approach to more time periods. For example, with 3 or 4 periods:

```python
# Example: 4 time periods (creates models: 4→3, 3→2, 2→1, 1→0)
features_mapping = {
    "spends": {
        ("spends_lag_1", 1): ["age_lag1", "income_lag1"],
        ("spends_lag_2", 2): ["age_lag2", "income_lag2"],
        ("spends_lag_3", 3): ["age_lag3", "income_lag3"],
        ("spends_lag_4", 4): ["age_lag4", "income_lag4"],
    }
}

# Different targets can have different numbers of periods
features_mapping = {
    "spends": {
        ("spends_lag_1", 1): ["covariates_lag1"],
        ("spends_lag_2", 2): ["covariates_lag2"],
        ("spends_lag_3", 3): ["covariates_lag3"],  # 3 periods for spends
    },
    "revenue": {
        ("revenue_lag_1", 1): ["covariates_lag1"],
        ("revenue_lag_2", 2): ["covariates_lag2"],  # 2 periods for revenue
    }
}
```

### When to use CUPED vs CUPAC:

**CUPED** is better when:
- You have **one strong covariate** (correlation > 0.5 with target)
- Need **simple, interpretable** results
- **Small to medium** sample sizes

**Multilevel CUPAC** is better when:
- You have **multiple time periods** of historical data
- Want **maximum variance reduction** through temporal modeling
- **Multiple covariates** available for each period
- **Large sample sizes** available

### Important considerations:
1. **Multilevel modeling**: CUPAC creates separate models for each time period transition
2. **No data leakage**: All covariates and targets must be from before the experiment
3. **Balance check**: Ensure covariates are balanced across treatment groups
4. **Missing data**: All methods require complete covariate data